In [1]:
from packages import *

# Torsionsweicher Träggerrost

Das folgende Beispiel ist aus @marti_baustatik_2014 entnommen. Dieses dient als einführendes Beispiel in die Modellierung von Trägerrosten. In der @fig-marti_rost_gr ist der Grundriss des Trägerrosts dargestellt. Es handelt sich um insgesamt 16 torsionsweiche Stäbe, welche an den Enden eingespannt sind.

![Grundriss des torsionsweichen Trägerrosts](../imgs/marti_traegerrost_gr.png){#fig-marti_rost_gr}

Im Beispiel wird eine analytische Lösung zur Traglast aufgezeigt. Das Ziel ist es, mittels dem Federmodell eine numerische Lösung des unteren Grenzwerts der Traglast zu erhalten.







## Analytische Lösung nach Marti

Die Analytische Lösung basiert auf dem Traglastverfahren. Mittels einem zulässigen Mechanismus wird ein oberer Grenzwert der Traglast hergeleitet. In der @fig-marti_mechanismus sind zwei Stäbe dargestellt. Aus symmetriegründen lässt sich der obere Grenzwert des gesamten Trägerrosts anhand dieser Darstellung ermitteln.

![Mechanismus eines Stabs des Trägerrosts](../imgs/marti_traegerrost_mechanismus.png){#fig-marti_mechanismus}



Die äussere Arbeit $W_a$ des dargestellten Systems in @fig-marti_mechanismus beträgt dabei für die am Rand gelegenen Stäbe (Punkte 2'45):

$$
W_{a,2'45} = 4 \cdot \left( Q \cdot \frac{1}{3} + Q \cdot \frac{1}{9} \right)
$$

Und für die Innenträger:

$$
W_{a,123} = 4 \cdot \left( Q \cdot 1 + Q \cdot \frac{1}{3} \right)
$$


Die innere Arbeit beträgt:

$$
W_i = 8 \cdot \left(M_u' + M_u\right) \cdot \left(\frac{1}{3l} +\frac{1}{9l}\right)
$$


Durch das abschliessende Gleichsetzen der Arbeiten und das Lösen nach $Q$ folgt der obere Grenzwert der Traglast zu:

$$
Q = \frac{M_u + M_u'}{2l}
$$


Die Plastizitätskontrolle ist in der @fig-marti_plastizitaetskontrolle gezeigt. Dabei wird eine Lastverteilung von je einer Hälfte $Q$ in $x$ und $y$ Richtung angenommen.

![Plastizitätskontrolle anhand der Schnittgrössen](../imgs/marti_traegerrost_schnittgroessen.png){#fig-marti_plastizitaetskontrolle}

Aus der Plastizitätskontrolle geht heraus, dass der Biegewiderstand nirgends überschritten wird, somit deckt sich der untere und obere Grenzwert der Traglast, sprich die Traglast $Q_u$ wurde gefunden.


$$
Q_u = \frac{M_u + M_u'}{2l}
$$



## Analytische Lösung Pascal 

![Trägerrost nachgezeichnet](../imgs/rost_pascal.png){#fig-rost_pascal}

Dies wird kurz verifiziert


### Oberer Grenzwert

In [2]:
l, Q, M_u, M_apos_u = sp.symbols("l, Q, M_u, M'u")

![Schnitt A-A](../imgs/aa_pascal.png){#fig-schnitt_aa}



![Schnitt B-B](../imgs/bb_pascal.png){#fig-schnitt_bb}

In [3]:
W_A_aa = 2 * (Q * (1/3) + Q)
W_A_bb = 2 * (Q * (1/9) + Q * (1/3))

W_A = 2 * (W_A_aa + W_A_bb)

W_A

7.11111111111111*Q

In [4]:
W_I_aa = 2 * (M_apos_u * 1/(3*l) + M_u * (1/(3*l)))
W_I_bb = 2 * (M_apos_u * 1/(9*l) + M_u * (1/(9*l)))

W_I = 4 *  (W_I_aa + W_I_bb)

W_I

32*M'u/(9*l) + 32*M_u/(9*l)

In [5]:
Q_upper = sp.solve(sp.Eq(W_A, W_I), Q)[0]
Q_upper

0.5*(M'u + M_u)/l

Der obere Grenzwert stimmt mit der Lösung von Marti überein.

### Unterer Grenzwert

Zunächst gilt es die Lastaufteilung im Trägerrost zu bestimmen. Gemäss Marti wird die Einzellast je zur Hälfte in $x$ und $y$ Richtung abgetragen. Aufgrund der Geometrie des Trägerrost ist dies jedoch nicht korrekt.

![Lastaufteilung des Trägerrosts im Grundriss](../imgs/lastaufteilung_pascal.png){#fig-lastaufteilung_traegerrost}

Aus der Lastaufteilung geht hervor, dass der Schnitt B-B die massgebenden Lasten aufweist. Der untere Grenzwert der Traglast wird dementsprechend an diesem bestimmt. Dargestellt ist der Schnitt B-B in der @fig-bb_lastaufteilung.

![Lastaufteilung im Schnitt B-B](../imgs/bb_unterer_grenzwert.png){#fig-bb_lastaufteilung}

Mittels Gleichgewicht der Momente um den Punkt $A$ folgt der untere Grenzwert der Traglast zu:

In [6]:
sum_M_A = 3/4 * Q * l + 1/2 * Q * 3*l  - M_u - M_apos_u
Q_lower = sp.solve(sum_M_A, Q)[0]

Q_lower.evalf(2)

0.44*(M'u + M_u)/l

Es zeigt sich, dass sich der obere und untere Grenzwert nicht decken. Die Traglast befindet sich in diesen Grenzen.

## Numerische Lösung

Das erwähnte Ziel, mittels dem Federmodell den unteren Grenzwert der Traglast zu berechnen, wird folgend dargestellt.
Dabei werden zunächst numerische Werte definiert.



In [7]:
l = 1*m 
M_u = 100*kN*m
M_apos_u = 100*kN*m

Q_lower = 0.44444444444*(M_u + M_apos_u) /l
Q_upper = 0.5*(M_u + M_apos_u) /l

put_out()

$$
\begin{aligned}
l & = 1 \ \mathrm{m} \quad & M_{u} & = 100 \ \mathrm{kN} \cdot \mathrm{m} \quad & M'_{u} & = 100 \ \mathrm{kN} \cdot \mathrm{m} \\ 
Q_{lower} & = 88.89 \ \mathrm{kN} \quad & Q_{upper} & = 100.0 \ \mathrm{kN} \quad &  
 \end{aligned}
$$

Der Trägerrost wurde mittels dem Federmodell nachmodelliert. Dabei sind biegestarre, jedoch torsionsweiche Stäbe mittels Stabendgelenken gekoppelt. Dargestellt in der @fig-rfem_marti_geom.

![Statisches Modell in RFEM des Trägerrosts](../imgs/marti_rfem_geom.png){#fig-rfem_marti_geom}

![Gelenkdefinition in RFEM des Trägerrosts](../imgs/marti_rfem_gelenk.png){#fig-rfem_marti_gelenk}

Den Gelenken wurde die Defintion gemäss @fig-rfem_marti_gelenk hinterlegt. Dabei wurde der Biegewiderstand in positiver und negativer Dimension angesetzt, ab dem Erreichen des Biegewiderstands fliesst das Gelenk. Dies führt dazu, dass sich die Biegemomente umlagern, sprich der Biegewiderstand auch in Trägermitte erreicht werden kann.

![Biegemomente in RFEM des Trägerrosts](../imgs/marti_rfem_res.png){#fig-rfem_marti_res}

Mit der Belastung aus dem unteren Grenzwert stellen sich exakt die Extremwerte der Biegemomente $M_u$ und $M'_u$ in den innen liegenden Stäben ein. Das Federmodell liefert den exakten unteren Grenzwert der Traglast. Dargestell in der @fig-rfem_marti_res.